# --- Day 10: Pipe Maze ---
You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:
```
| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
```
Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:
```
.....
.F-7.
.|.|.
.L-J.
.....
```
If the animal had entered this loop in the northwest corner, the sketch would instead look like this:
```
.....
.S-7.
.|.|.
.L-J.
.....
```
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:
```
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
```
In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:
```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:
```
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
```
If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:
```
.....
.S-7.
.|.|.
.L-J.
.....
```
You can count the distance each tile in the loop is from the starting point like this:
```
.....
.012.
.1.3.
.234.
.....
```
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:
```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here are the distances for each tile on that loop:
```
..45.
.236.
01.78
14567
23...
```
**Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?**

In [1]:
from utilities import get_lines
from collections import namedtuple

In [2]:
lines = get_lines('input')

In [3]:
def get_S_location(lines):
    for i, line in enumerate(lines):
        if 'S' in line:
            row = i
    col = lines[row].index('S')
    return row,col

In [4]:
pipes = {'|':'NS',
         '-':'EW',
         'L':'NE',
         'J':'NW',
         '7':'SW',
         'F':'SE',
         'S':'start'
        }

In [5]:
max_row = len(lines)-1
max_col = len(lines[0])-1
max_row, max_col

(139, 139)

In [6]:
def is_valid_loc(i,j):
    if (0<=i<=max_row)&(0<=j<=max_col):
        return True
    return False

In [7]:
class Tile:
    def __init__(self, row, col, pipe):
        self.row = row
        self.col = col
        self.pipe = pipe
        self.N = False
        self.E = False
        self.S = False
        self.W = False
        self.is_start = False
        self.step = 0
        self.from_dir = None
    
    Loc = namedtuple('Loc','row col pipe')
    
    def __repr__(self):
        return f'({self.row}, {self.col}, {self.pipe})'
    
    
    def check_NS(self): #  |
        if self.pipe=='|':
            if is_valid_loc(self.row-1,self.col):
                self.N = True
                return True
            if is_valid_loc(self.row+1,self.col):
                self.S = True
                return True
        return False
    
    def check_EW(self): #  -
        if self.pipe=='-':
            if is_valid_loc(self.row,self.col+1):
                self.E = True
                return True
            if is_valid_loc(self.row,self.col-1):
                self.W = True
                return True
        return False
    
    def check_NE(self): #  L
        if self.pipe=='L':
            if is_valid_loc(self.row-1,self.col):
                self.N = True
                return True
            if is_valid_loc(self.row,self.col+1):
                self.E = True
                return True
        return False
    
    def check_NW(self): #  J
        if self.pipe=='J':
            if is_valid_loc(self.row-1,self.col):
                self.N = True
                return True
            if is_valid_loc(self.row,self.col-1):
                self.W = True
                return True
        return False
    
    def check_SW(self): #  7
        if self.pipe=='7':
            if is_valid_loc(self.row+1,self.col):
                self.S = True
                return True
            if is_valid_loc(self.row,self.col-1):
                self.W = True
                return True
        return False
    
    def check_SE(self): #  F
        if self.pipe=='F':
            if is_valid_loc(self.row+1,self.col):
                self.S = True
                return True
            if is_valid_loc(self.row,self.col+1):
                self.E = True
                return True
        return False
    
    def set_start(self): #  S
        if self.pipe=='S':
            self.is_start = True
            return 1
        return 0
    
    def step_N(self):
        return (self.row-1,self.col)
    
    def step_E(self):
        return (self.row,self.col+1)
    
    def step_S(self):
        return (self.row+1,self.col)
    
    def step_W(self):
        return (self.row,self.col-1)

Ok, we have a start location, now we have to check around to find out which way to go.

In [8]:
def find_first_step(start_tile):
    #  check the tile to the north, does it have a connection to the south?
    r, c = start_tile.step_N() 
    if is_valid_loc(r,c):
        next_tile = Tile(r,c,lines[r][c])
        if next_tile.check_NS()|next_tile.check_SE()|next_tile.check_SW():
            next_tile.from_dir = 'S'
            return next_tile
        
    #  check the tile to the east, does it have a connection to the west?
    r, c = start_tile.step_E()
    if is_valid_loc(r,c):
        next_tile = Tile(r,c,lines[r][c])
        if next_tile.check_EW()|next_tile.check_NW()|next_tile.check_SW():
            next_tile.from_dir = 'W'
            return next_tile
        
    #  check the tile to the south, does it have a connection to the north?
    r, c = start_tile.step_S()
    if is_valid_loc(r,c):
        next_tile = Tile(r,c,lines[r][c])
        if next_tile.check_NS()|next_tile.check_NW()|next_tile.check_NW():
            next_tile.from_dir = 'N'
            return next_tile
    
    #  no need to check to the west because one of the above HAS to return true
    return 'error'

In [9]:
def get_next_direction(from_dir,pipe_dirs):
    '''
    from_dir: char 
    pipe_dirs: str of two chars, one of which is the from_dir
    '''
    return str(set(pipe_dirs)-set(from_dir))[2]

In [10]:
def get_next_loc(curr_tile, next_direction):
    if next_direction=='N':
        r, c, = curr_tile.step_N()
    elif next_direction=='E':
        r, c = curr_tile.step_E()
    elif next_direction=='S':
        r, c = curr_tile.step_S()
    elif next_direction=='W':
        r, c = curr_tile.step_W()
    else:
        r, c = -1, -1
    return (r,c)

In [11]:
def get_came_from(next_direction):
    #  return the opposite on the compass rose
    compass = 'NESW'
    i = compass.index(next_direction)
    return compass[(i+2)%4]

In [12]:
def get_next_tile(curr_tile):
    # print('\n\ncame from direction ',curr_tile.from_dir)
    # don't need to check that direction
    # print('pipe ',curr_tile.pipe, 'directions ', pipes[curr_tile.pipe])
    next_direction = get_next_direction(curr_tile.from_dir, pipes[curr_tile.pipe])
    # print('\tnext direction ',next_direction)
    r, c = get_next_loc(curr_tile, next_direction)
    # print(r,c)
    next_tile = Tile(r,c,lines[r][c])
    next_tile.step = curr_tile.step+1
    next_tile.from_dir = get_came_from(next_direction)
    return next_tile

In [13]:
start_tile = Tile(*get_S_location(lines),'S')
start_tile.set_start()
print(f'start: {start_tile}')

curr_tile = find_first_step(start_tile)
curr_tile.step = 1
print(f'first tile: {curr_tile}')

path_tiles = [start_tile, curr_tile]

next_tile = get_next_tile(curr_tile)
print(f'next tile: {next_tile}')

steps = 0
while next_tile.pipe!='S':
# for i in range(5):
    path_tiles.append(next_tile)
    # print(curr_tile, curr_tile.step)
    curr_tile = next_tile
    next_tile = get_next_tile(curr_tile)
    steps+=1

print(f'returning to {next_tile} took {steps} steps')


start: (102, 118, S)
first tile: (103, 118, |)
next tile: (104, 118, |)
returning to (102, 118, S) took 14008 steps


In [14]:
print(f'So the farthest step away is {steps//2 + 1}')

So the farthest step away is 7005
